# importantion des library

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

warnings.filterwarnings("ignore")

# Lecture du fichier *titanic.csv*

In [2]:
def charger_et_preparer_donnees():
    df = pd.read_csv("Titanic.csv")
    df.columns = [
        "age",
        "classe_travail",
        "education",
        "nombre_education",
        "etat_civil",
        "occupation",
        "Lien_parente",
        "race",
        "sexe",
        "heures_par_semaine",  # Corrected the dot to underscore
        "pays_natal",
        "salaire",
    ]
    # Créer l'échantillon (premiers 10%)
    taille_echantillon = int(len(df) * 0.1)
    df_echantillon = df.iloc[:taille_echantillon]
    return df, df_echantillon

# Étude exploratoire (5 pts)

In [3]:
df, df_echantillon = charger_et_preparer_donnees()
df.head()

,age,classe_travail,education,nombre_education,etat_civil,occupation,Lien_parente,race,sexe,heures_par_semaine,pays_natal,salaire
0,90,?,HS-grad,9,Veuf,?,Pas_dans_famille,Blanc,Feminin,40,United-States,<=50K
1,82,Privee,HS-grad,9,Veuf,Exec-managerial,Pas_dans_famille,Blanc,Feminin,18,United-States,<=50K
2,66,?,Some-college,10,Veuf,?,Celibataire,Noir,Feminin,40,United-States,<=50K
3,54,Privee,7th-8th,4,Divorce,Machine-op-inspct,Celibataire,Blanc,Feminin,40,United-States,<=50K
4,41,Privee,Some-college,10,Separe,Prof-specialty,Un_enfant,Blanc,Feminin,40,United-States,<=50K


**Note** Apres la lecture de donnee on voit un petit soucis avec les noms des colomns donc on va les traiter

In [4]:
df_echantillon.shape

(3256, 12)

In [5]:
df.shape

(32561, 12)

# 1. Combien de personnes travaillent-elles dans le privé ?

In [6]:
travail_privee = len(df[df["classe_travail"] == "Privee"])

travail_privee

22696

**Reponse:** il y'a 22696 personnes qui travaillent dans le prive.

# 2. Quels sont les pays d’origine des femmes dans ce jeux de données ?

In [7]:
# 2. Pays d'origine des femmes
pays_femmes = df[df["sexe"] == "Feminin"]["pays_natal"].unique()
print(f"\n2. Pays d'origine des femmes:\n{pays_femmes}")


2. Pays d'origine des femmes:
['United-States' '?' 'Trinadad&Tobago' 'Philippines' 'Canada'
 'Holand-Netherlands' 'Honduras' 'South' 'Hungary' 'Mexico' 'Taiwan'
 'Cambodia' 'Columbia' 'Guatemala' 'Japan' 'Cuba' 'Puerto-Rico'
 'Dominican-Republic' 'England' 'Germany' 'Iran' 'France' 'Scotland'
 'Portugal' 'Nicaragua' 'El-Salvador' 'Italy' 'Ecuador' 'Peru' 'India'
 'Jamaica' 'Laos' 'Poland' 'China' 'Outlying-US(Guam-USVI-etc)' 'Haiti'
 'Hong' 'Vietnam' 'Ireland' 'Yugoslavia' 'Greece' 'Thailand']


# 3. Quel est le pourcentage d’hommes ayant un niveau éducatif HS-grad ?

In [8]:
total_hommes = len(df[df["sexe"] == "Masculin"])
pourcentage_homme = (
    len(df[(df["sexe"] == "Masculin") & (df["education"] == "HS-grad")]) / total_hommes
)

print(
    f"Le pourcentage d'hommes ayant un niveau éducatif HS-grades est: {pourcentage_homme * 100}"
)

Le pourcentage d'hommes ayant un niveau éducatif HS-grades est: 32.634235888022026


# 4. Quelles sont les attributs pour lesquels manquent de valeurs ? Combien de valeurs manquent par attribut ?

In [9]:
valeurs_manquantes = df.isin(["?"]).sum()
valeurs_manquantes = valeurs_manquantes[valeurs_manquantes > 0]
print("\n4. Valeurs manquantes par attribut:")
print(valeurs_manquantes)
print(f"\nAttribut avec le plus de valeurs manquantes: {valeurs_manquantes.idxmax()}")
print(
    f"Attribut avec le moins de valeurs manquantes (>0): {valeurs_manquantes[valeurs_manquantes > 0].min()}"
)


4. Valeurs manquantes par attribut:
classe_travail    1836
occupation        1843
pays_natal         583
dtype: int64

Attribut avec le plus de valeurs manquantes: occupation
Attribut avec le moins de valeurs manquantes (>0): 583


In [10]:
df.replace("?", np.nan, inplace=True)
df_echantillon.replace("?", np.nan, inplace=True)

In [11]:
df.isna().sum()

age                      0
classe_travail        1836
education                0
nombre_education         0
etat_civil               0
occupation            1843
Lien_parente             0
race                     0
sexe                     0
heures_par_semaine       0
pays_natal             583
salaire                  0
dtype: int64

# Les colonnes avec valeurs manquantes
1. classe_travail avec **1836** valeurs manquantes 
2. occupation avec **1843** valeurs manquantes
3. pays-natal avec **583** valeurs manquantes

## Quels attributs ont le nombre de valeurs manquantes maximal ? Et minimal (mais > 0) ?

### L'attribut avec le nombre maximal des valeurs manquantes est: *Ocupation*

### L'attribut avec le nombre minimal des valeurs manquantes est: *pays-natal*

# 5. Quelles sont les valeurs la plus élevée et la moins élevée de l’attribut nombre d’années d’éducation

In [12]:
max_valeur = df.iloc[:, 3].max()
min_valeur = df.iloc[:, 3].min()
print(f" La valeur maximal est: {max_valeur}")
print(f" La valeur minimal est: {min_valeur}")

 La valeur maximal est: 16
 La valeur minimal est: 1


# Statistiques et probabilités (22 pts)
Pour la suite, nous considérons le jeux de données complet ainsi qu’un échantillon dont la taille est 10% de la taille <br>
complète (3256 lignes). Pour des raisons de déterminisme, on prendra les premiers 10% des enregistrements (lignes 1
à 3256).

In [13]:
df_echantillon.shape

(3256, 12)

In [14]:
# visualisation de l'echantillon
df_echantillon.head()

,age,classe_travail,education,nombre_education,etat_civil,occupation,Lien_parente,race,sexe,heures_par_semaine,pays_natal,salaire
0,90,NaN,HS-grad,9,Veuf,NaN,Pas_dans_famille,Blanc,Feminin,40,United-States,<=50K
1,82,Privee,HS-grad,9,Veuf,Exec-managerial,Pas_dans_famille,Blanc,Feminin,18,United-States,<=50K
2,66,NaN,Some-college,10,Veuf,NaN,Celibataire,Noir,Feminin,40,United-States,<=50K
3,54,Privee,7th-8th,4,Divorce,Machine-op-inspct,Celibataire,Blanc,Feminin,40,United-States,<=50K
4,41,Privee,Some-college,10,Separe,Prof-specialty,Un_enfant,Blanc,Feminin,40,United-States,<=50K


# 1. Extraire les 3 valeurs du nombre d’années d’éducation les plus fréquentes. Correspondent-elles à un intervalle complet (ex. 8, 9 et 10) ?

In [15]:
education_plus_frequent = df_echantillon.iloc[:, 3].value_counts()
education_plus_frequent.head(3)

nombre_education
13    845
9     731
10    552
Name: count, dtype: int64

### Rep: Les trois valeurs les plus frequentes sur la colonnes *nombre_education* sont: 13,9,10, donc l'interval ne correspond pas a l'intervalle donnee ici.

# 2. Moyenne des nombre d’années d’éducation ? Et médiane ?

In [16]:
moyenne_nombre_annee_education = df_echantillon.iloc[:, 3].mean()

mediane_nombre_annee_education = df_echantillon.iloc[:, 3].median()


print(
    f"La moyenne de nombre d'annees d'education est: {moyenne_nombre_annee_education}"
)
print(
    f"La mediane de nombre d'annees d'education est: {mediane_nombre_annee_education}"
)

La moyenne de nombre d'annees d'education est: 11.430896805896806
La mediane de nombre d'annees d'education est: 12.0


# Quels pays ont les moyennes des âges de leurs ressortissants les plus élevées ?

In [17]:
age_moyen_par_pays = (
    df_echantillon.groupby("pays_natal")["age"].mean().sort_values(ascending=False)
)
print("\n3. Pays avec les moyennes d'âge les plus élevées:")
print(age_moyen_par_pays)


3. Pays avec les moyennes d'âge les plus élevées:
pays_natal
Ecuador               90.000000
Hungary               69.000000
Yugoslavia            66.000000
Scotland              62.000000
Jamaica               60.000000
Columbia              57.000000
Poland                50.750000
Greece                50.000000
China                 49.555556
Trinadad&Tobago       49.500000
France                49.000000
Honduras              47.000000
El-Salvador           46.000000
Italy                 45.600000
Guatemala             45.500000
Germany               45.071429
Vietnam               44.833333
India                 44.533333
Cuba                  44.000000
Canada                43.882353
United-States         43.559584
Philippines           42.952381
Iran                  42.875000
Japan                 42.625000
England               42.100000
Puerto-Rico           42.000000
Nicaragua             40.000000
Hong                  40.000000
South                 38.181818
Mexico    

In [18]:
df_echantillon.groupby("pays_natal")["age"].mean().sort_values(ascending=False)

pays_natal
Ecuador               90.000000
Hungary               69.000000
Yugoslavia            66.000000
Scotland              62.000000
Jamaica               60.000000
Columbia              57.000000
Poland                50.750000
Greece                50.000000
China                 49.555556
Trinadad&Tobago       49.500000
France                49.000000
Honduras              47.000000
El-Salvador           46.000000
Italy                 45.600000
Guatemala             45.500000
Germany               45.071429
Vietnam               44.833333
India                 44.533333
Cuba                  44.000000
Canada                43.882353
United-States         43.559584
Philippines           42.952381
Iran                  42.875000
Japan                 42.625000
England               42.100000
Puerto-Rico           42.000000
Nicaragua             40.000000
Hong                  40.000000
South                 38.181818
Mexico                38.062500
Haiti                 37.0000

# 4  Quelle % des femmes ayant une maîtrise ont un salaire >50K ? Quelle est ce % chez les hommes ?

In [19]:
def calc_pourcentage_salaire(donnees, genre):
    maitrise = donnees[(donnees["sexe"] == genre) & (donnees["education"] == "Masters")]
    if len(maitrise) == 0:
        return 0
    return len(maitrise[maitrise["salaire"] == ">50K"]) / len(maitrise) * 100


pct_femmes_maitrise = calc_pourcentage_salaire(df, "Feminin")
pct_hommes_maitrise = calc_pourcentage_salaire(df, "Masculin")
print(f"\n4. Pourcentage >50K avec maîtrise:")
print(f"Femmes: {pct_femmes_maitrise:.2f}%")
print(f"Hommes: {pct_hommes_maitrise:.2f}%")


4. Pourcentage >50K avec maîtrise:
Femmes: 33.40%
Hommes: 65.71%


# 5. Quelle % des personnes ne vivant pas en famille ont un diplôme universitaire (baccalauréat, maîtrise ou doctorat) ?

In [20]:
# 5. Pourcentage de diplômés universitaires hors famille
diplomes_univ = ["Bachelors", "Masters", "Doctorate"]
hors_famille = df[df["Lien_parente"] == "Pas_dans_famille"]
pct_diplomes = (
    len(hors_famille[hors_famille["education"].isin(diplomes_univ)])
    / len(hors_famille)
    * 100
)
print(
    f"\n5. Pourcentage de personnes hors famille avec diplôme universitaire: {pct_diplomes:.2f}%"
)


5. Pourcentage de personnes hors famille avec diplôme universitaire: 27.43%


# 6. Quelle race a la proportion la plus élevée de salaires ≤50K ? Qu’en est il pour la valeur >50K ?

In [21]:
def analyser_proportions_race(dataset):
    
    # Calcul des proportions pour salaires ≤50K
    race_inf_50K = df[df['salaire'] == '<=50K'].groupby('race').size() / \
                   df.groupby('race').size() * 100
    
    # Calcul des proportions pour salaires >50K
    race_sup_50K = df[df['salaire'] == '>50K'].groupby('race').size() / \
                   df.groupby('race').size() * 100
    
    # Identifier les races avec proportions maximales
    race_max_inf_50k = race_inf_50K.idxmax()
    race_max_sup_50k = race_sup_50K.idxmax()
    
    # Affichage des résultats
    print(f"Race avec la proportion la plus élevée de salaires ≤50K (échantillon) : "
          f"{race_max_inf_50k} ({race_inf_50K[race_max_inf_50k]:.2f}%)")
    print(f"Race avec la proportion la plus élevée de salaires >50K (échantillon) : "
          f"{race_max_sup_50k} ({race_sup_50K[race_max_sup_50k]:.2f}%)")

analyser_proportions_race(df_echantillon)

Race avec la proportion la plus élevée de salaires ≤50K (échantillon) : Autre (90.77%)
Race avec la proportion la plus élevée de salaires >50K (échantillon) : Asiatique (26.56%)


In [22]:
race_inf_50K = df_echantillon[df_echantillon['salaire'] == '<=50K'].groupby('race').size() / df_echantillon.groupby('race').size() * 100

race_sup_50K = df_echantillon[df_echantillon['salaire'] == '>50K'].groupby('race').size() / df_echantillon.groupby('race').size() * 100

race_max_inf_50k = race_inf_50K.idxmax()
race_max_sup_50k = race_sup_50K.idxmax()

print(f"Race avec la proportion la plus élevée de salaires ≤50K (échantillon) : {race_max_inf_50k} ({race_inf_50K[race_max_inf_50k]:.2f}%)")
print(f"Race avec la proportion la plus élevée de salaires >50K (échantillon) : {race_max_sup_50k} ({race_sup_50K[race_max_sup_50k]:.2f}%)")


Race avec la proportion la plus élevée de salaires ≤50K (échantillon) : Autre (52.94%)
Race avec la proportion la plus élevée de salaires >50K (échantillon) : Asiatique (73.15%)


# 7. Existe-t-il une différence dans les salaires des hommes mariées et les hommes ne vivant pas en famille ? Dans quel sens va-t-elle ?

In [23]:
# Filtrer les données pour les hommes uniquement
men_df_echantillon = df_echantillon[df_echantillon['sexe'] == 'Masculin']

# Séparer les hommes mariés et ceux ne vivant pas en famille
marie_homme = men_df_echantillon[men_df_echantillon['etat_civil'].str.contains('Marie', na=False)]
pas_famile_homme = men_df_echantillon[men_df_echantillon['Lien_parente'] == 'Pas_dans_famille']

# Calculer la répartition des salaires pour chaque groupe
salaire_marie_distribution = marie_homme['salaire'].value_counts(normalize=True) * 100
salaire_non_famille = pas_famile_homme['salaire'].value_counts(normalize=True) * 100

# Afficher les résultats
print("Hommes mariés :")
print(f"Proportion de salaires ≤50K : {salaire_marie_distribution.get('<=50K', 0):.2f} %")
print(f"Proportion de salaires >50K : {salaire_marie_distribution.get('>50K', 0):.2f} %\n")

print("Hommes ne vivant pas en famille :")
print(f"Proportion de salaires ≤50K : {salaire_non_famille.get('<=50K', 0):.2f} %")
print(f"Proportion de salaires >50K : {salaire_non_famille.get('>50K', 0):.2f} %")


Hommes mariés :
Proportion de salaires ≤50K : 16.28 %
Proportion de salaires >50K : 83.72 %

Hommes ne vivant pas en famille :
Proportion de salaires ≤50K : 48.35 %
Proportion de salaires >50K : 51.65 %


**Les hommes mariés ont une proportion beaucoup plus élevée de salaires >50K par rapport aux hommes ne vivant pas en famille. Cela indique que les hommes mariés tendent à avoir des salaires plus élevés que leurs homologues ne vivant pas en famille.**

In [24]:
# Question 7 : Différence de salaires hommes mariés vs célibataires
def analyse_salaires_hommes():
    hommes = df[df["sexe"] == "Masculin"]
    hommes_maries = hommes[hommes["etat_civil"].isin(["Marie"])]
    hommes_celibataires = hommes[hommes["Lien_parente"] == "Celibataire"]

    salaires_maries = (hommes_maries["salaire"] == ">50K").mean()
    salaires_celibataires = (hommes_celibataires["salaire"] == ">50K").mean()
    print(f"Le salaire de Marie est: {salaires_maries}")
    print(f"Le salaire de celibataires est: {salaires_celibataires}")

    return {
        "salaires_maries": salaires_maries,
        "salaires_celibataires": salaires_celibataires,
        "difference": salaires_maries - salaires_celibataires,
    }


analyse_salaires_hommes()

Le salaire de Marie est: nan
Le salaire de celibataires est: 0.13383838383838384


{'salaires_maries': nan,
 'salaires_celibataires': 0.13383838383838384,
 'difference': nan}

# 8. Quelle est la variance des âges dans notre jeu de données complet ? Quelle est sa valeur pour l’échantillon ?

In [25]:
# variance de donnees compet
variance_age_complete = df["age"].var()

# variance de l'echantillon
variance_df_echantillon = df_echantillon["age"].var()

print("8. Variance des âges:")
print(
    f"   Variance des âges de notre jeu de données complet : {variance_age_complete:.2f}"
)
print(
    f"   Variance de l'échantillon des âges de notre jeu de données : {variance_df_echantillon:.2f}"
)

8. Variance des âges:
   Variance des âges de notre jeu de données complet : 186.06
   Variance de l'échantillon des âges de notre jeu de données : 149.91


# 9. Parmi les années d’éducation et l'âge, quel attribut a plus d’influence sur la valeur du salaire ? Dit autrement, lequel des deux est plus corrélé avec le salaire ? Pour y répondre, il faudrait coder l’attribut ordinal salaire par de nombres, par ex., ≤50K par 1 et >50K par 2.

In [26]:
 # Encoder le salaire (1 pour <=50K, 2 pour >50K)
 df_echantillon['salaire_encode'] = df_echantillon['salaire'].map({'<=50K': 1, '>50K': 2})
 
 # Convertir education en nombres d'années (approximation)
 education_mapping = {
     'Preschool': 1, '1st-4th': 4, '5th-6th': 6, '7th-8th': 8,
     '9th': 9, '10th': 10, '11th': 11, '12th': 12, 'HS-grad': 12,
     'Some-college': 14, 'Bachelors': 16, 'Masters': 18, 'Doctorate': 21
 }
 df_echantillon['education_annees'] = df_echantillon['education'].map(education_mapping)
 
 # corr_age = df_echantillon['age'].corr(df_echantillon['salaire_encode'])
 # corr_education = df_echantillon['education_annees'].corr(df_echantillon['salaire_encode'])

# Créer une matrice de corrélation incluant salaire_encode
df_echantillon[['age', 'education_annees', 'salaire_encode']].corr()['salaire_encode']


age                 0.210755
education_annees    0.326183
salaire_encode      1.000000
Name: salaire_encode, dtype: float64

les correlation:
1. **Âge et salaire (corrélation : 0.210755)**
2. **Années d'éducation et salaire (corrélation : 0.326183)** <br>
**En conclusion:** Le nombre d'années d'éducation semble jouer un rôle plus important que l'âge dans la détermination des salaires. Cela met en évidence l'importance de l'éducation pour obtenir des revenus plus élevés, bien que l'âge ait également un impact mineur


In [27]:
df_echantillon.select_dtypes(include='int').corr()['salaire_encode']

age                   0.210755
nombre_education      0.345756
heures_par_semaine    0.218557
salaire_encode        1.000000
Name: salaire_encode, dtype: float64

# 10. Est-ce que l’âge est un facteur d’influence important pour le salaire des femmes ? Répondez en comparant les ages moyens dans les groupes de femmes à salaires ≤50K et >50K, respectivement.

In [28]:
# Filtrer les données pour les femmes uniquement
women_df_echantillon = df_echantillon[df_echantillon['sexe'] == 'Feminin']

# Calculer l'âge moyen pour les groupes de salaires
average_age_by_salary = women_df_echantillon.groupby('salaire')['age'].mean()

# Afficher les résultats
print("Âge moyen des femmes par groupe de salaire :")
print(f"Femmes avec un salaire ≤50K : {average_age_by_salary.get('<=50K', 0):.2f} ans")
print(f"Femmes avec un salaire >50K : {average_age_by_salary.get('>50K', 0):.2f} ans")


Âge moyen des femmes par groupe de salaire :
Femmes avec un salaire ≤50K : 38.46 ans
Femmes avec un salaire >50K : 43.16 ans


# 11. Quelle est la différence entre les écarts types des heures par semaine pour le jeux complet et pour l’échantillon ?

In [29]:
ecartype_complet = df["heures_par_semaine"].std()

ecartype_echantillon = df_echantillon["heures_par_semaine"].std()

ecartype_difference = ecartype_complet - ecartype_echantillon

print(
    f"les écarts types des heures par semaine pour le jeux complet  : {ecartype_complet:.2f}"
)
print(
    f"les écarts types des heures par semaine pour l’échantillon: {ecartype_echantillon:.2f}"
)
print(
    f"La différence entre les écarts types des heures par semaine pour le jeux complet et pour l’échantillon: {ecartype_difference:.2f}"
)

les écarts types des heures par semaine pour le jeux complet  : 12.35
les écarts types des heures par semaine pour l’échantillon: 11.72
La différence entre les écarts types des heures par semaine pour le jeux complet et pour l’échantillon: 0.62


# 3. Tests d’hypothèses (9 pts)
Dans cette partie, nous allons considérer le jeu de données complet comme étant notre « population » et notre
attention sera portée sur l’échantillon des 1ers 3256 lignes qui nous servira comme base pour la validation de diverses
hypothèses. On assume aussi que l’attribut nombre d’heures par semaine suit une loi normale.
1. On émet l’hypothèse que la moyenne des heures par semaine sur le jeu de données complet n’est pas plus élevée
que 40 hrs/sem. En vous basant sur l’échantillon, et en utilisant un écart type de la population égal à 12,35 ,
testez cette hypothèse à l’aide d’un t-test. Prenez un niveau de signification de 5%.
• N.B. Pour des raisons pédagogiques, on ignore la valeur effective de la moyenne pour le jeu complet :
ce qu’on veut tester est si la valeur observée sur l’échantillon est suffisamment extrême pour permettre
de rejeter l’hypothèse H0 que vous aurez formulée.
2. On émet une autre hypothèse : les femmes travaillent, en moyenne, plus d’heures que la moyenne hypothétique
de 40 hrs/sem. En vous basant uniquement sur la partie pertinente de l’échantillon, testez cette hypothèse à
l’aide d’un t-test. Prenez un niveau de signification de 10%.
3. Finalement, existe-t-il un écart significatif (à 5%) entre le salaire moyen de la population et celui des femmes ?
Pour y répondre, utilisez le codage suggéré par la question 9. Avec ce codage, l’écart type de la population est
arrondi à 0,43.

# $z= \frac{\bar{x} - \mu_0}{s/\sqrt{n}}$


In [30]:
from scipy import stats

def tests_hypotheses(df, df_echantillon):
    try:
        # 1. Test si moyenne d'heures par semaine ≤ 40
        ecart_type_pop = 12.35
        moyenne_echantillon = df_echantillon['heures_par_semaine'].mean()
        n = len(df_echantillon)
        
        stat_t = (moyenne_echantillon - 40) / (ecart_type_pop / np.sqrt(n))
        valeur_p = stats.norm.cdf(stat_t)
        
        print("1. Test d'hypothèse pour moyenne d'heures ≤ 40:")
        print(f"Moyenne de l'échantillon: {moyenne_echantillon:.2f} heures")
        print(f"Statistique t: {stat_t:.4f}")
        print(f"Valeur p: {valeur_p:.4f}")
        print(f"Rejet H0 à 5%: {valeur_p < 0.05}")
        
        # 2. Test si les femmes travaillent plus de 40 heures en moyenne
        heures_femmes = df_echantillon[df_echantillon['sexe'] == 'Feminin']['heures_par_semaine']
        if len(heures_femmes) == 0:
            print("\nAvertissement: Aucune donnée pour les femmes dans l'échantillon")
        else:
            stat_t, valeur_p = stats.ttest_1samp(heures_femmes, 40)
            valeur_p_unilateral = valeur_p / 2 if stat_t > 0 else 1.0
            
            print("\n2. Test d'hypothèse pour femmes travaillant > 40 heures:")
            print(f"Moyenne des heures (femmes): {heures_femmes.mean():.2f}")
            print(f"Statistique t: {stat_t:.4f}")
            print(f"Valeur p (unilatéral): {valeur_p_unilateral:.4f}")
            print(f"Rejet H0 à 10%: {valeur_p_unilateral < 0.10}")
        
        # 3. Test de différence de salaire entre population et femmes
        ecart_type_pop = 0.43
        df_echantillon['salaire_code'] = df_echantillon['salaire'].map({'<=50K': 1, '>50K': 2})
        if df_echantillon['salaire_code'].isnull().any():
            raise ValueError("Valeurs inattendues dans la colonne 'salaire'")
        
        df['salaire_code'] = df['salaire'].map({'<=50K': 1, '>50K': 2})
        echantillon_femmes = df_echantillon[df_echantillon['sexe'] == 'Feminin']['salaire_code']
        
        if len(echantillon_femmes) == 0:
            print("\nAvertissement: Aucune donnée salariale pour les femmes")
        else:
            moyenne_pop = df['salaire_code'].mean()
            moyenne_femmes = echantillon_femmes.mean()
            
            stat_t = (moyenne_femmes - moyenne_pop) / (ecart_type_pop / np.sqrt(len(echantillon_femmes)))
            valeur_p = 2 * (1 - stats.norm.cdf(abs(stat_t)))
            
            print("\n3. Test de différence de salaire pour les femmes:")
            print(f"Moyenne population: {moyenne_pop:.4f}")
            print(f"Moyenne femmes: {moyenne_femmes:.4f}")
            print(f"Statistique t: {stat_t:.4f}")
            print(f"Valeur p: {valeur_p:.4f}")
            print(f"Différence significative à 5%: {valeur_p < 0.05}")
            
    except Exception as e:
        print(f"Erreur lors des tests d'hypothèses: {str(e)}")

def main():
    try:
        print("\n=== Tests d'Hypothèses ===")
        tests_hypotheses(df, df_echantillon)
    except Exception as e:
        print(f"Erreur dans le programme principal: {str(e)}")

if __name__ == "__main__":
    main()



=== Tests d'Hypothèses ===
1. Test d'hypothèse pour moyenne d'heures ≤ 40:
Moyenne de l'échantillon: 44.44 heures
Statistique t: 20.5149
Valeur p: 1.0000
Rejet H0 à 5%: False

2. Test d'hypothèse pour femmes travaillant > 40 heures:
Moyenne des heures (femmes): 39.07
Statistique t: -2.1616
Valeur p (unilatéral): 1.0000
Rejet H0 à 10%: False

3. Test de différence de salaire pour les femmes:
Moyenne population: 1.2408
Moyenne femmes: 1.5135
Statistique t: 16.8151
Valeur p: 0.0000
Différence significative à 5%: True


In [31]:
# Second code working

In [32]:
def tests_hypotheses(df, df_echantillon):
    
    try:
        # 1. Test si moyenne d'heures par semaine ≤ 40
        ecart_type_pop = 12.35
        moyenne_echantillon = df_echantillon['heures_par_semaine'].mean()
        n = len(df_echantillon)
        
        # Calcul de la statistique t
        stat_t = (moyenne_echantillon - 40) / (ecart_type_pop / np.sqrt(n))
        print(stat_t)
        # Pour un test unilatéral à gauche
        valeur_p = stats.norm.cdf(stat_t)
        
        print("1. Test d'hypothèse pour moyenne d'heures ≤ 40:")
        print(f"Moyenne de l'échantillon: {moyenne_echantillon:.2f} heures")
        print(f"Statistique t: {stat_t:.4f}")
        print(f"Valeur p: {valeur_p:.4f}")
        print(f"Rejet H0 à 5%: {valeur_p < 0.05}")
        
        # 2. Test si les femmes travaillent plus de 40 heures en moyenne
        heures_femmes = df_echantillon[df_echantillon['sexe'] == 'Feminin']['heures_par_semaine']
        if len(heures_femmes) == 0:
            print("\nAvertissement: Aucune donnée pour les femmes dans l'échantillon")
        else:
            stat_t, valeur_p = stats.ttest_1samp(heures_femmes, 40)
            
            print("\n2. Test d'hypothèse pour femmes travaillant > 40 heures:")
            print(f"Moyenne des heures (femmes): {heures_femmes.mean():.2f}")
            print(f"Statistique t: {stat_t:.4f}")
            print(f"Valeur p (unilatéral): {valeur_p/2:.4f}")
            print(f"Rejet H0 à 10%: {valeur_p/2 < 0.10}")
        
        # 3. Test de différence de salaire entre population et femmes
        ecart_type_pop = 0.43
        
        # Codage salaire (≤50K: 1, >50K: 2)
        df_echantillon['salaire_code'] = df_echantillon['salaire'].map({'<=50K': 1, '>50K': 2})
        df['salaire_code'] = df['salaire'].map({'<=50K': 1, '>50K': 2})
        echantillon_femmes = df_echantillon[df_echantillon['sexe'] == 'Feminin']['salaire_code']
        
        if len(echantillon_femmes) == 0:
            print("\nAvertissement: Aucune donnée salariale pour les femmes")
        else:
            moyenne_pop = df['salaire_code'].mean()
            moyenne_femmes = echantillon_femmes.mean()
            
            stat_t = (moyenne_femmes - moyenne_pop) / (ecart_type_pop / np.sqrt(len(echantillon_femmes)))
            valeur_p = 2 * (1 - stats.norm.cdf(abs(stat_t)))
            
            print("\n3. Test de différence de salaire pour les femmes:")
            print(f"Moyenne population: {moyenne_pop:.4f}")
            print(f"Moyenne femmes: {moyenne_femmes:.4f}")
            print(f"Statistique t: {stat_t:.4f}")
            print(f"Valeur p: {valeur_p:.4f}")
            print(f"Différence significative à 5%: {valeur_p < 0.05}")
            
    except Exception as e:
        print(f"Erreur lors des tests d'hypothèses: {str(e)}")

def main():
    try:
        print("\n=== Tests d'Hypothèses ===")
        tests_hypotheses(df, df_echantillon)
    except Exception as e:
        print(f"Erreur dans le programme principal: {str(e)}")

if __name__ == "__main__":
    main()


=== Tests d'Hypothèses ===
20.51488209610187
1. Test d'hypothèse pour moyenne d'heures ≤ 40:
Moyenne de l'échantillon: 44.44 heures
Statistique t: 20.5149
Valeur p: 1.0000
Rejet H0 à 5%: False

2. Test d'hypothèse pour femmes travaillant > 40 heures:
Moyenne des heures (femmes): 39.07
Statistique t: -2.1616
Valeur p (unilatéral): 0.0155
Rejet H0 à 10%: True

3. Test de différence de salaire pour les femmes:
Moyenne population: 1.2408
Moyenne femmes: 1.5135
Statistique t: 16.8151
Valeur p: 0.0000
Différence significative à 5%: True
